### Importação e Download das Bibliotecas

###### Importa as bibliotecas necessárias para processamento de texto, modelagem e extração dos parâmetros, incluindo o download do pacote de stopwords do NLTK.

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from datetime import timedelta
import pandas as pd
import unicodedata
import dateparser
import pickle
import string
import nltk
import re

# Baixar stopwords em português
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
# Definir as stopwords
stop_words = set(stopwords.words('portuguese'))

### Carregamento dos Dados

###### Carrega o dataset de intenções a partir de um arquivo JSON, exibindo as primeiras linhas para verificação.

In [32]:
file_path = "../../../data/processed/intents.json"
df = pd.read_json(file_path)

In [33]:
df.head()

,id,intent,utterance
0,1,consulta_normativa,Quais são as Instruções da CVM sobre Administr...
1,2,consulta_normativa,Gostaria de ver as Deliberações relacionadas a...
2,3,consulta_normativa,Existem Ofícios Circulares sobre Agentes Fiduc...
3,4,consulta_normativa,Quais são as Leis e Decretos que falam sobre A...
4,5,consulta_normativa,Há alguma Nota Explicativa sobre Analistas de ...


### Processamento dos Dados

###### Realiza a limpeza e transformação do texto, removendo pontuações, acentuações e stopwords, para preparar os dados para a vetorização e modelagem.

In [34]:
# Função para pré-processar o texto: remove pontuações, stopwords e normaliza os caracteres.
def preprocess_text(text):
    # Converte para minúsculas
    text = text.lower()
   
    # Remove pontuações
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove espaços em branco extras
    text = text.strip()

    # Remove stopwords
    filtered_words = filter(lambda word: word not in stop_words, text.split())
    text = " ".join(filtered_words)
    
    # Remove acentos
    text = ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'))
    
    return text

In [35]:
# Aplica o pré-processamento às entradas
df['processed_utterance'] = df['utterance'].apply(preprocess_text)

### Vetorização do Texto

###### Converte o texto pré-processado em uma matriz de TF-IDF para ser usada como entrada no modelo de machine learning.

In [36]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_utterance'])

### Divisão dos Dados em Treino e Teste

###### Divide os dados em conjuntos de treino e teste para a avaliação do modelo.

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, df['intent'], test_size=0.2, random_state=42)

### Treinamento do Modelo

###### Treina um modelo de Naive Bayes Multinomial usando os dados de treino.

In [38]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

### Predição e Avaliação

###### Realiza predições no conjunto de teste e avalia a performance do modelo usando métricas de classificação.

In [39]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

                    precision    recall  f1-score   support

consulta_normativa       1.00      1.00      1.00        21
            outros       1.00      1.00      1.00        19

          accuracy                           1.00        40
         macro avg       1.00      1.00      1.00        40
      weighted avg       1.00      1.00      1.00        40



### Mapeamento de Tags

###### Cria um dicionário para converter as tags identificadas no texto pré-processado para seus nomes corretamente formatados

In [40]:
tag_mapping = {
    'administradores carteiras': 'Administradores de Carteiras',
    'agencias classificacao risco credito': 'Agências de Classificação de Risco de Crédito',
    'agentes fiduciarios': 'Agentes Fiduciários',
    'alerta': 'Alerta',
    'analistas valores mobiliarios': 'Analistas de Valores Mobiliários',
    'assessores investimento': 'Assessores de Investimento',
    'ato declaratorio': 'Ato Declaratório',
    'atuacao irregular': 'Atuação Irregular',
    'audiencia publica': 'Audiência Pública',
    'auditor independente': 'Auditor Independente',
    'bdr': 'BDR',
    'cadastro participantes regulados': 'Cadastro de Participantes Regulados',
    'clubes investimento': 'Clubes de Investimento',
    'companhia': 'Companhia',
    'comunicado mercado': 'Comunicado ao Mercado',
    'concurso premio': 'Concurso/Prêmio',
    'consultores valores mobiliarios': 'Consultores de Valores Mobiliários',
    'convenio': 'Convênio',
    'coronavirus': 'Coronavirus',
    'corretora': 'Corretora',
    'crowdfunding': 'Crowdfunding',
    'decisao colegiado': 'Decisão do Colegiado',
    'deliberacao': 'Deliberação',
    'educacao financeira': 'Educação Financeira',
    'evento': 'Evento',
    'fundos investimento': 'Fundos de Investimento',
    'fundos investimento direitos creditorios': 'Fundos de Investimento em Direitos Creditórios',
    'fundos investimento participacoes': 'Fundos de Investimento em Participações',
    'fundos investimento imobiliarios': 'Fundos de Investimento Imobiliários',
    'gestao institucional': 'Gestão Institucional',
    'indenizacao': 'Indenização',
    'infraestrutura mercado': 'Infraestrutura do Mercado',
    'insider trading': 'Insider Trading',
    'intermediarios': 'Intermediários',
    'investidores nao residentes': 'Investidores Nao Residentes',
    'julgamento': 'Julgamento',
    'julgamento insider': 'Julgamento_Insider',
    'mercados organizados': 'Mercados Organizados',
    'normas contabeis': 'Normas Contábeis',
    'nota': 'Nota',
    'ofertas publicas': 'Ofertas Publicas',
    'oficio circular': 'Ofício Circular',
    'ouvidoria': 'Ouvidoria',
    'parecer orientacao': 'Parecer de Orientação',
    'pesquisa': 'Pesquisa',
    'planejamento estrategico': 'Planejamento Estratégico',
    'pld ftp': 'PLD/FTP',
    'processo eletronico': 'Processo Eletrônico',
    'protocolo digital': 'Protocolo Digital',
    'publicacao': 'Publicação',
    'ritos cvm': 'Ritos CVM',
    'sandbox regulatorio': 'Sandbox Regulatório',
    'securitizadoras': 'Securitizadoras',
    'sistema governanca gestao cvm': 'Sistema de governança e gestão da CVM',
    'suitability': 'Suitability',
    'suspensao': 'Suspensão',
    'tecnologia informacao': 'Tecnologia da Informação',
    'termo compromisso': 'Termo de Compromisso',
    'termo compromisso insider': 'Termo_Compromisso_Insider'
}

### Extração de Datas

###### Define uma função para identificar e extrair datas mencionadas no texto. A função lida com intervalos de datas, datas absolutas e expressões temporais relativas.

In [41]:
def extract_dates(text):
    
    # Converte o texto para minúsculas e corrige palavras que podem estar sem acentuação
    text = text.lower()
    text = text.replace('mes', 'mês').replace('ate', 'até')

    extracted_dates = []

    # Verifica se o texto contém intervalos de datas nos formatos "de DD/MM/AAAA até DD/MM/AAAA" ou "entre DD/MM/AAAA e DD/MM/AAAA"
    interval_match = re.search(r'de (\d{1,2}/\d{1,2}/\d{2,4}) até (\d{1,2}/\d{1,2}/\d{2,4})', text, re.IGNORECASE) or \
                     re.search(r'entre (\d{1,2}/\d{1,2}/\d{2,4}) e (\d{1,2}/\d{1,2}/\d{2,4})', text, re.IGNORECASE)
    

    if interval_match:
        # Se um intervalo for encontrado, extrai as duas datas, processa e converte para o formato 'YYYY-MM-DD'
        start_date_str, end_date_str = interval_match.groups()
        start_date = dateparser.parse(start_date_str, languages=['pt'])
        end_date = dateparser.parse(end_date_str, languages=['pt'])
        extracted_dates.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    
    else:
        # Se não há intervalo, a função procura por datas individuais ou expressões de tempo relativas
        date_matches = re.findall(
            r'\b(?:\d{1,2}/\d{1,2}/\d{2,4}|\d{1,2} de \w+ de \d{4}|hoje|ontem|anteontem|semana passada|mês passado|ano passado|há \d+ (?:dias|semanas|meses|anos))\b',
            text, re.IGNORECASE
        )

        for date_str in date_matches:
            parsed_date = dateparser.parse(date_str, languages=['pt'])

            if parsed_date:
                # Para expressões de tempo relativas, calcula o intervalo de datas correspondente
                if "semana passada" in date_str.lower():
                    start_of_week = parsed_date - timedelta(days=parsed_date.weekday() + 1)
                    end_of_week = start_of_week + timedelta(days=6) 
                    extracted_dates.append((start_of_week.strftime('%Y-%m-%d'), end_of_week.strftime('%Y-%m-%d')))
                
                elif "mês passado" in date_str.lower():
                    start_of_month = parsed_date.replace(day=1)
                    end_of_month = (parsed_date.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1)
                    extracted_dates.append((start_of_month.strftime('%Y-%m-%d'), end_of_month.strftime('%Y-%m-%d')))
                
                elif "ano passado" in date_str.lower():
                    start_of_year = parsed_date.replace(month=1, day=1)
                    end_of_year = parsed_date.replace(month=12, day=31)
                    extracted_dates.append((start_of_year.strftime('%Y-%m-%d'), end_of_year.strftime('%Y-%m-%d')))
                
                else:
                    # Para datas absolutas ou outras expressões relativas que não representam intervalos, converte a data para o formato padrão
                    extracted_dates.append(parsed_date.strftime('%Y-%m-%d'))

    # Retorna uma lista de datas ou intervalos de datas encontrados, no formato 'YYYY-MM-DD'
    return extracted_dates if extracted_dates else None

### Extração de Parâmetros

###### Define uma função que extrai informações estruturadas, como tipo de documento, tags, órgão regulador e datas, a partir do texto processado. A função utiliza expressões regulares para identificar padrões específicos dentro do texto.

In [42]:
def extract_parameters(utterance, processed_utterance):
    # Expressões regulares para identificar tipos de documentos, tags e órgão regulador.
    document_type_regex = r'(instrucoes|pareceres orientacao|deliberacoes|decisoes conjuntas|oficios circulares|leis decretos|notas explicativas)'
    tags_regex = r'(administradores carteiras|agencias classificacao risco credito|agentes fiduciarios|alerta|analistas valores mobiliarios|assessores investimento|ato declaratorio|atuacao irregular|audiencia publica|auditor independente|bdr|cadastro participantes regulados|clubes investimento|companhia|comunicado mercado|concurso premio|consultores valores mobiliarios|convenio|coronavirus|corretora|crowdfunding|decisao colegiado|deliberacao|educacao financeira|evento|fundos investimento|fundos investimento direitos creditorios|fundos investimento participacoes|fundos investimento imobiliarios|gestao institucional|indenizacao|infraestrutura mercado|insider trading|intermediarios|investidores nao residentes|julgamento|julgamento insider|mercados organizados|normas contabeis|nota|ofertas publicas|oficio circular|ouvidoria|parecer orientacao|pesquisa|planejamento estrategico|pld ftp|processo eletronico|protocolo digital|publicacao|ritos cvm|sandbox regulatorio|securitizadoras|sistema governanca gestao cvm|suitability|suspensao|tecnologia informacao|termo compromisso|termo compromisso insider)'
    entity_regex = r'cvm'

    # Realiza a busca no texto processado para identificar o tipo de documento, tags e órgão regulador.
    document_type = re.search(document_type_regex, processed_utterance)
    tags = re.search(tags_regex, processed_utterance)
    entity = re.search(entity_regex, processed_utterance)

    # Extrai as datas do texto sem processamento
    dates = extract_dates(utterance)

    # Mapeia as tags para os nomes corretamente formatados
    if tags:
        tags = tag_mapping.get(tags.group(0), tags.group(0))

    # Retorna os parâmetros extraídos em um dicionário estruturado
    return {
        'document_type': document_type.group(0) if document_type else None,
        'tags': tags if tags else None,
        'entity': entity.group(0) if entity else None,
        'dates': dates if dates else None
    }

### Predição e Extração de Parâmetros

###### Define uma função que processa uma entrada, realiza a predição da intenção usando o modelo treinado e, se aplicável, extrai parâmetros adicionais a partir do texto.

In [43]:
def predict_and_extract(utterance, model, vectorizer):
    # Pré-processa a entrada para garantir que o texto esteja no formato adequado para o modelo
    processed_utterance = preprocess_text(utterance)

    # Converte o texto pré-processado em uma matriz vetorial TF-IDF
    vectorized_utterance = vectorizer.transform([processed_utterance])

    # Utiliza o modelo treinado para prever a intenção da entrada
    prediction = model.predict(vectorized_utterance)[0]
    
    # Se a intenção for "consulta_normativa", extrai parâmetros adicionais
    if prediction == "consulta_normativa":
        parameters = extract_parameters(utterance, processed_utterance)
        print(f"Intenção: {prediction}")
        print(f"Parâmetros Extraídos: {parameters}")
    else:
        # Se a intenção não for relevante para o escopo do projeto, apenas imprime a intenção prevista
        print(f"Intenção: {prediction} (fora do escopo do projeto)")

### Exportação do Modelo

###### Salva o modelo treinado e o vetorizador TF-IDF em arquivos para reutilização futura.


In [44]:
# Salva o modelo
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Salva o vetor TF-IDF
with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)